In [ ]:
import kagglehub
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
base_dir = '/kaggle/input/selfies-id-images-dataset/Selfies ID Images dataset'
train_dir = os.path.join(base_dir, '18_sets_ Caucasians')
validation_dir = os.path.join(base_dir, '11_sets_Hispanics')

img_height = img_width = 128
batch_size = 64

train_ds = image_dataset_from_directory(train_dir, seed=123, image_size=(64, 64), batch_size=batch_size)
val_ds = image_dataset_from_directory(validation_dir, seed=123, image_size=(64, 64), batch_size=batch_size)


model = Sequential([
    Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    
    Conv2D(64, kernel_size=(3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    
    Flatten(),  # Приведение к плоской форме перед полносвязным слоем
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Финальный выходной слой
])


# Показываем сводку архитектуры модели
model.summary()

# Определяем критерии оптимизации и компиляции модели
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.002), metrics=['accuracy'])

# Добавляем раннюю остановку, чтобы избежать переобучения
early_stopping = EarlyStopping(monitor='val_loss', patience=6)

# Начинаем процесс обучения
history = model.fit(train_ds, validation_data=val_ds, epochs=30, callbacks=[early_stopping])

# Строим график изменения потери (Loss) на этапах обучения и валидации
plt.figure(figsize=(20, 10))
plt.subplot(2, 3, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Потеря (Loss)')
plt.xlabel('Эпоха')
plt.ylabel('Ошибка')
plt.legend()

# График изменения точности (Accuracy)
plt.subplot(2, 4, 4)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Точность (Accuracy)')
plt.xlabel('Эпоха')
plt.ylabel('Точность')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Функция для прогноза одиночного изображения
def predict_image(image_path):
    img = tf.keras.utils.load_img(image_path, target_size=(img_height, img_width))  # Загружаем изображение нужного размера
    img_array = tf.keras.utils.img_to_array(img)  # Преобразуем в массив
    img_array = tf.expand_dims(img_array, 0)  # Создаём батч (одиночное изображение)
    prediction = model.predict(img_array)[0][0]  # Прогнозируем класс
    return 'Подделка!' if prediction > 0.5 else 'Реальная!'

# Использовать функцию predict_image для любого нового изображения
new_image_path = input("Введите ссылку на изображение(минимальная ширина фото - 128 пикселей): ")
result = predict_image(new_image_path)
print(f'Результат прогноза: {result}')